<a href="https://colab.research.google.com/github/karenl7/AA548-spr2024/blob/dev/homework/hw1_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class DynUnicycle:

    def state_derivative(self, state, control):
        '''
        Computes x_dot where x_dot = f(x, u)
        '''
        pass

    def euler_step(self, state, control, dt):
        pass

    def analytic_step(self, state, control, dt):
        pass

    def RK4_step(self, state, control, dt):
        pass

    def linearize_continuous_time_analytic(self, state, control) -> tuple:
        pass

def discrete_time_simulate(discrete_time_dyn, initial_state, control_sequence):
    pass

def linearize(function_name, state, control):
    '''function_name takes in only (state, control)'''
    pass


# Problem 1

The plotting code is provided below. But you need to fill out some functions first in order to run it.
- `state_derivative`
- `euler_step`
- `analytic_step`
- `RK4_step`
- `discrete_time_simulate`


In [ ]:
# generating initial state and control sequence
initial_state = jnp.array([0, 0, jnp.pi/4, 2.])  # initial state, size [4,] array
def generate_control_sequence(dt, horizon=5):
    N = round(horizon / dt)
    w_sequence = jnp.sin(jnp.linspace(0, jnp.pi, N))  # w varies sinusoidally over time
    return jnp.stack([w_sequence, jnp.zeros(N)], 1)  # a is zero (constant speed), size [T, 2] array

In [ ]:
unicycle = DynUnicycle()

plt.figure(figsize=(15,4))

dt_array = [0.01, 0.1, 0.5]

for (i,dt) in enumerate(dt_array):

    controls = generate_control_sequence(dt, 5)

    plt.subplot(1,3,i+1)
    traj = discrete_time_simulate(jax.jit(lambda s,c: unicycle.analytic_step(s,c,dt)), initial_state, controls)
    plt.plot(traj[:,0], traj[:,1], label="Analytic hold")
    plt.scatter(traj[:,0], traj[:,1], s=2)


    traj = discrete_time_simulate(jax.jit(lambda s,c: unicycle.euler_step(s,c,dt)), initial_state, controls)
    plt.plot(traj[:,0], traj[:,1], label="Euler")
    plt.scatter(traj[:,0], traj[:,1], s=2)

    traj = discrete_time_simulate(jax.jit(lambda s,c: unicycle.RK4_step(s,c,dt)), initial_state, controls)
    plt.plot(traj[:,0], traj[:,1], label="RK4")
    plt.scatter(traj[:,0], traj[:,1], s=2)

    plt.grid()
    plt.legend()

    plt.title("dt=%.3f"%dt)

# plt.savefig(...) # you can save fig

# Problem 2

In [ ]:
x0 = jnp.array([0., 0., jnp.pi/4, 2.])
u0 = jnp.array([0.1, 1.])


### 2(c)  

You need to fill in code for `linearize` function.
And also `linearize_continuous_time_analytic` to test your analytic solution.

Some code to print out values from linearizing the continuous time dynamics about a point

In [ ]:
decimal_places = 2

A, B, C = unicycle.linearize_continuous_time_analytic(x0, u0)
print("Linearization from analytic expression")
print("A\n", round(A, decimal_places))
print("B\n", round(B, decimal_places))
print("C\n", round(C, decimal_places))

print("\n\n")

A, B, C = linearize(unicycle.state_derivative, x0, u0)
print("Linearization using autodiff ")
print("A\n", round(A, decimal_places))
print("B\n", round(B, decimal_places))
print("C\n", round(C, decimal_places))


### 2(d)

Some code to print out values from linearizing the zero-order hold dynamics about a point

In [ ]:
decimal_places = 2

A, B, C = linearize(lambda s,c: unicycle.analytic_step(s, c, dt), x0, u0)
print("Linearization using autodiff ")
print("A\n", round(A, decimal_places))
print("B\n", round(B, decimal_places))
print("C\n", round(C, decimal_places))


## Problem 3

In [ ]:
def cart_pole(state, control):
    pass

def cart_pole_linearized(state, control, state_eq, control_eq):
    pass

def linearization_error(state, control, state_eq, control_eq):
    pass


In [ ]:
# x, theta, xdot, thetadot
state_eq = jnp.array([0., jnp.pi, 0.0, 0.0])
control_eq = jnp.zeros(1)
states = jnp.array([[0,0.99*jnp.pi,0,0],
                    [0,0.9*jnp.pi,0,0],
                    [0,0.85*jnp.pi,0,0],
                    [0,0.5*jnp.pi,0,0],
                    [0,0,0,0],
                    [1,jnp.pi,0,0]])
controls = jnp.array([[-0.],
                      [-10.],
                      [0.],
                      [0.],
                      [0.],
                      [10.]])

errors = jax.vmap(linearization_error, in_axes=[0, 0, None, None])(states, controls, state_eq, control_eq)


In [ ]:
for i, e in enumerate(errors):
    print(f"State = {np.around(states[i], decimals=3)}^T")
    print(f"Input = {np.around(controls[i], decimals=3)}")
    print("Linearization error = %.3f"%e)
    print("Norm of f(x,u) = %.3f\n"%np.linalg.norm(cart_pole(states[i], controls[i])))